In [1]:
# Importació de llibreries necessàries per al processament de textos, modelatge de temes,
# classificació supervisada i anàlisi de mètriques
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import numpy as np

In [2]:
# Descàrrega de recursos de NLTK per a la tokenització, lematització i etiquetatge gramatical
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


In [3]:
# Funció per eliminar emojis del text utilitzant una expressió regular que filtra caràcters Unicode corresponents
def eliminar_emojis(texto):
    if isinstance(texto, str):
        patron_emojis = re.compile(pattern="[" 
            "\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF"
            "\U0001F680-\U0001F6FF" "\U0001F700-\U0001F77F"
            "\U0001F780-\U0001F7FF" "\U0001F800-\U0001F8FF"
            "\U0001F900-\U0001F9FF" "\U0001FA00-\U0001FA6F"
            "\U0001FA70-\U0001FAFF" "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251" "]+", flags=re.UNICODE)
        return patron_emojis.sub(r'', texto)
    return texto

# Inicialització del lematitzador basat en WordNet
lemmatizer = WordNetLemmatizer()

# Funció auxiliar que obté l'etiqueta gramatical (part-of-speech) de cada paraula per millorar la lematització
def obtener_pos_tag(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Funció que aplica la lematització al text:
# Tokenitza el text, aplica la lematització amb POS, i retorna el text recomposat
def lematizar_texto(texto):
    palabras = nltk.word_tokenize(texto)
    palabras_lematizadas = [lemmatizer.lemmatize(p, obtener_pos_tag(p)) for p in palabras]
    return ' '.join(palabras_lematizadas)


In [4]:
# Carrega del conjunt de dades SMS des d’una font en línia
url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])

# Conversió de les etiquetes de classe: 'ham' es codifica com 0 i 'spam' com 1
df['label'] = df['label'].replace({'ham': 0, 'spam': 1})

# Aplicació de la neteja i normalització del text:
# Eliminació d'emojis i posterior lematització
df['message'] = df['message'].apply(eliminar_emojis).apply(lematizar_texto)

# Definició d'una llista de paraules buides (stopwords) que s'exclouran en el model de característiques
stopwords = ['a', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
             'is', 'it', 'for', 'with', 'that', 'this', 'as', 'was', 'be',
             'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
             'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not']

# Separació de les dades en variables independents (X_text) i dependents (y)
X_text = df["message"]
y = df["label"]


In [5]:
# Definició dels valors de k per al nombre de temes en el model LDA
valors_k = [10]
resultats = []

In [6]:

# Importació addicional (no necessària) d’un altre model de boosting
from sklearn.ensemble import GradientBoostingClassifier

# Configuració del mètode de validació creuada estratificada (5 particions)
# Es garanteix una distribució equilibrada de les classes en cada partició
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Bucle principal d'entrenament i validació per a cada valor de k
for k in valors_k:
    aucs = []  # Llista per emmagatzemar les puntuacions AUC per cada partició
    for train_index, test_index in skf.split(X_text, y):
        # Separació de les dades entre conjunt d'entrenament i de prova
        X_train_raw, X_test_raw = X_text.iloc[train_index], X_text.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Vectorització dels textos mitjançant una representació bag-of-words amb un límit de 10.000 característiques
        vectorizer = CountVectorizer(max_features=10000, stop_words=stopwords)
        X_train_counts = vectorizer.fit_transform(X_train_raw)
        X_test_counts = vectorizer.transform(X_test_raw)

        # Aplicació del model LDA per a la reducció de dimensionalitat i extracció de temes
        lda = LatentDirichletAllocation(n_components=k, random_state=42)
        X_train_topics = lda.fit_transform(X_train_counts)
        X_test_topics = lda.transform(X_test_counts)

        # Inicialització i entrenament del model de classificació XGBoost
        model = XGBClassifier(
            max_depth=6,            # Profunditat màxima dels arbres de decisió
            n_estimators=500,       # Nombre total d’arbres
            learning_rate=0.01,     # Taxa d’aprenentatge
            random_state=42         # Semilla per a la reproduïbilitat
        )
        model.fit(X_train_topics, y_train)

        # Predicció de probabilitats per a la classe positiva i càlcul de la mètrica AUC-ROC
        probs = model.predict_proba(X_test_topics)[:, 1]
        auc = roc_auc_score(y_test, probs)
        aucs.append(auc)

        # Impressió per indicar el progrés de la iteració
        print("iteració")

    # Emmagatzematge de la mitjana de les puntuacions AUC per aquest valor de k
    resultats.append({"k": k, "auc_roc_mitjana": np.mean(aucs)})

# Creació del DataFrame final amb els resultats obtinguts
df_resultats = pd.DataFrame(resultats)

iteració
iteració
iteració
iteració
iteració


In [7]:
df_resultats  # Visualització del resum de resultats

,k,auc_roc_mitjana
0,10,0.965705
